In [41]:
import os
import re
import nltk
from nltk.tokenize import sent_tokenize
import json

In [79]:
files = os.listdir("./books")
files = [file for file in files if file.endswith(".txt") and not "split" in file]
print(files)

already_saved_files = os.listdir("./books/saves")
already_saved_files = [file for file in already_saved_files if file.endswith(".txt")]

for file in files:
    if file in already_saved_files:
        print(f"Skipping {file}")
        continue
    
    with open(f"./books/{file}", "r") as f:
        lines = f.readlines()
    
    lines_out = []
    sub_txt = ""
    chapter_texts = set()
    for line in lines:
        if line.strip().lower().startswith("chapter"):
            # handle chapter
            if sub_txt:
                lines_out.extend(sent_tokenize(sub_txt))
                sub_txt = ""
            
            chapter_text = line.strip()
            # seek forward until we get an empty line
            next_line = lines[lines.index(line) + 1]
            print(chapter_text, "|", next_line)
            while not next_line.strip() == "":
                chapter_text += " " + next_line.strip()
                next_line = lines[lines.index(next_line) + 1]
            chapter_text += "\n"
            chapter_texts.add(chapter_text)
            print(chapter_text)
            lines_out.append(chapter_text)



        else:
            if any(line.strip() in chapter_text for chapter_text in chapter_texts):
                continue
            sub_txt += line.strip() + " "

    if sub_txt:
        lines_out.extend(sent_tokenize(sub_txt))
    

    # Split lines longer than 150 characters
    lines_out_out = []
    for idx, line in enumerate(lines_out):
        if len(line) > 150:
            # Define potential split points in order of preference
            split_markers = ['. ', '; ', ', ', ' and ', ' but ', ' or ', ' ', '\n']
            text_parts = []
            current_text = line
            
            while len(current_text) > 150:
                split_found = False
                # Try each marker in order until we find one within the 150 char limit
                for marker in split_markers:
                    # Find the last occurrence of the marker within first 150 chars
                    pos = current_text[:150].rfind(marker)
                    if pos != -1:
                        # Split at the marker
                        text_parts.append(current_text[:pos + len(marker)].strip())
                        current_text = current_text[pos + len(marker):].strip()
                        split_found = True
                        break
                
                # If no marker found, force split at 150
                if not split_found:
                    text_parts.append(current_text[:150])
                    current_text = current_text[150:].strip()
            
            if current_text:  # Add remaining text
                text_parts.append(current_text)
            
            # Replace original line with split lines
            lines_out_out.extend(text_parts)
        else:
            lines_out_out.append(line)

    # save original text
    with open(f"./books/saves/{file}", "w") as f:
        f.write("\n".join(lines))
    
    # save split text
    with open(f"./books/{file}", "w") as f:
        f.write("\n".join(lines_out_out))



['Around-The-World-In-80-Days.txt', 'Harry-Potter-01-and-the-Sorcerors-Stone.txt', 'Harry-Potter-02-and-the-Chamber-of-Secrets.txt', 'Harry-Potter-03-and-The-Prisoner-of-Azkaban.txt', 'Harry-Potter-04-and-the-Goblet-of-Fire.txt', 'Harry-Potter-05-and-the-Order-of-the-Phoenix.txt', 'Harry-Potter-06-and-the-Half-Blood-Prince.txt', 'Harry-Potter-07-and-the-Deathly-Hallows.txt', 'Moby-Dick.txt', 'The-Hobbit.txt', 'Twenty-Thousand-Leagues-Under-The-Sea.txt']
Skipping Around-The-World-In-80-Days.txt
Skipping Harry-Potter-01-and-the-Sorcerors-Stone.txt
Skipping Harry-Potter-02-and-the-Chamber-of-Secrets.txt
Skipping Harry-Potter-03-and-The-Prisoner-of-Azkaban.txt
Skipping Harry-Potter-04-and-the-Goblet-of-Fire.txt
Skipping Harry-Potter-05-and-the-Order-of-the-Phoenix.txt
Skipping Harry-Potter-06-and-the-Half-Blood-Prince.txt
Skipping Harry-Potter-07-and-the-Deathly-Hallows.txt
Skipping Moby-Dick.txt
Skipping The-Hobbit.txt
Skipping Twenty-Thousand-Leagues-Under-The-Sea.txt
